# OOFの実装

In [12]:
import setting

from src import config
from src.common import util

In [24]:
import pandas as pd
import numpy as np
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.base import clone
from tqdm import tqdm

## load data

In [4]:
df_train = pd.read_csv(config.RAW_DATA_DIR / "train.csv")
df_test = pd.read_csv(config.RAW_DATA_DIR / "test.csv")

In [5]:
X_train_val = df_train.drop(columns=["target", "id"])
y_train_val = df_train["target"].values

X_test = df_test.drop(columns=["id"])

## oof

In [13]:
model = util.get_model("m001_hgbr")

In [7]:
from sklearn.model_selection import KFold

In [9]:
splitter = KFold(n_splits=5, random_state=0, shuffle=True)

In [23]:
df_train[["target"]]

,target
0,7.243043
1,8.203331
2,7.776091
3,6.957716
4,7.951046
...,...
299995,7.385215
299996,7.242617
299997,9.592487
299998,8.207951


In [42]:
df_oof = df_train[["target"]].copy()
df_oof["oof"] = np.nan
df_oof["fold"] = np.nan

splits = list(splitter.split(X_train_val, y_train_val))
for i, (train_idx, val_idx) in enumerate(tqdm(splits, desc="[OOF]")):
    _model = clone(model)
    _model.fit(X_train_val.iloc[train_idx, :], y_train_val[train_idx])
    y_pred = _model.predict(X_train_val.iloc[val_idx, :])
    df_oof.iloc[val_idx, df_oof.columns.get_loc("oof")] = y_pred
    df_oof.iloc[val_idx, df_oof.columns.get_loc("fold")] = i

[OOF]: 100%|██████████| 5/5 [00:15<00:00,  3.16s/it]


In [45]:
from sklearn.model_selection import cross_val_predict, check_cv
from sklearn.base import is_classifier

In [33]:
y_pred = cross_val_predict(model, X_train_val, y_train_val, cv=splitter)

In [36]:
(df_oof["oof"].values == y_pred).all()

True

cross_val_predictの結果とあっている

In [50]:
def oof(model, X_train_val, y_train_val, groups = None, cv = 5):
    X = np.asarray(X_train_val)
    y = np.asarray(y_train_val).ravel()

    df_oof = pd.DataFrame()
    df_oof["y_true"] = y
    df_oof["oof"] = np.nan
    df_oof["fold"] = np.nan

    cv = check_cv(cv, y, classifier=is_classifier(model))
    splits = list(cv.split(X, y, groups=groups))

    for i, (train_idx, val_idx) in enumerate(tqdm(splits, desc="[OOF]")):
        _model = clone(model)
        _model.fit(X[train_idx, :], y[train_idx])
        y_pred = _model.predict(X[val_idx, :])
        df_oof.iloc[val_idx, df_oof.columns.get_loc("oof")] = y_pred
        df_oof.iloc[val_idx, df_oof.columns.get_loc("fold")] = i

    df_oof["fold"] = df_oof["fold"].astype(np.uint8)

    return df_oof

In [51]:
df_oof = oof(model, X_train_val, y_train_val, cv=splitter)

[OOF]: 100%|██████████| 5/5 [00:17<00:00,  3.41s/it]


In [52]:
df_oof.head()

,y_true,oof,fold
0,7.243043,7.924785,1
1,8.203331,7.888473,3
2,7.776091,7.592755,1
3,6.957716,7.758425,3
4,7.951046,7.866027,4
